In [3]:
import numpy as np

# 假设的相机内参矩阵
intrinsics = np.array([601.1693115234375 , 0.,         637.8362426757812,
                    0. ,        600.8593139648438, 363.8018798828125,
                    0.,           0.,           1.        ]).reshape((3,3))
fx, fy = intrinsics[0,0], intrinsics[1,1]
cx, cy = intrinsics[0,2], intrinsics[1,2]

# 给定的像素坐标(u, v)和深度Z
u, v, Z = 820, 350, 0.627 # (0.18999086222955386, -0.014402337594503479, 0.627)
u, v, Z = 840, 507, 0.554 # (0.1863014618523999, 0.1320305048139165, 0.554)
u, v, Z = 523, 420, 0.606 # (-0.11575900786613977, 0.05667892633017958, 0.606)

# 计算XYZ坐标
X = (u - cx) * Z / fx
Y = (v - cy) * Z / fy

X, Y, Z

(-0.11575900786613977, 0.05667892633017958, 0.606)

In [8]:
import numpy as np
import open3d as o3d
 
def hand_eye_calib(points_cam, points_rob, vis=True):
    cam = o3d.geometry.PointCloud()
    rob = o3d.geometry.PointCloud()
    cam.points = o3d.utility.Vector3dVector(points_cam)
    rob.points = o3d.utility.Vector3dVector(points_rob)
    cam.paint_uniform_color([1, 0, 0])
    rob.paint_uniform_color([0, 1, 0])
    list_id_cam = list(range(len(points_cam)))
    list_id_rob = list(range(len(points_rob)))
    assert (len(list_id_cam) >= 3 and len(list_id_rob) >= 3)
    assert (len(list_id_rob) == len(list_id_rob))
    corr = np.zeros((len(list_id_cam), 2))
    corr[:, 0] = list_id_cam
    corr[:, 1] = list_id_rob
    print("zhanglei:", corr)
    hand_eye_p2p = o3d.pipelines.registration.TransformationEstimationPointToPoint()
    final_hand_eye = hand_eye_p2p.compute_transformation(cam, rob, o3d.utility.Vector2iVector(corr))
    print(final_hand_eye)
    
    if vis:
        o3d.visualization.draw_geometries([cam.transform(final_hand_eye), rob])
    return final_hand_eye
 

# points_cam = np.array(
#     [[-0.02075,  0.02597,  1.15615],
#      [0.15487, 0.05592,   1.11818],
#      [0.13379, -0.04346, 1.12400]] # camera 坐标系下的点
# )
points_cam = np.array(
    [[0.18999086222955386, -0.014402337594503479, 0.627],
     [0.1863014618523999, 0.1320305048139165, 0.554],
     [-0.11575900786613977, 0.05667892633017958, 0.606]]
)

# points_rob = np.array(
#     [[0.2449664800249867, -0.36223072817512053, 0.005815013802759242],
#      [0.42321621634623396, -0.3965202364977765, 0.0071650325018939726],
#      [0.4039400827566261, -0.29704639533281213, 0.007148857394119079]]
# )
points_rob = np.array([[0.5315284341408161, -0.31606744270290527, 0.12170995566217216],
[0.5290902012372901, -0.47521642506733847, 0.12233675564088844],
[0.23767231568711195, -0.3911261471407533, 0.11825402441267664]]
)

final_hand_eye = hand_eye_calib(points_cam, points_rob)

# cam
# [[-0.25047815, - 0.13900883,  1.50225439],
# [-0.02924966,  0.18728401,  1.46627832],
# [-0.38933743,  0.22643211,  1.53078687]]

# rob
# [[-0.10405,-0.60642,0.01483],
# [0.22889, -0.39027,0.01612],
# [0.25768,-0.75808,0.01991]]

# hand_eye_result
# [[0.03031916,  0.99942402,  0.01524406,  0.0198895],
#  [0.9824005,  -0.02698358, -0.18482733, -0.08615673],
#  [-0.18430953, 0.02057958, -0.98265278,  1.44772014],
#  [0.,          0.,          0.,          1.]]
hand_eye_m = final_hand_eye

# frame base
base_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0,0,0])
cam_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.2)
cam_frame.transform(np.reshape(hand_eye_m, (4,4)))
o3d.visualization.draw_geometries([base_frame, cam_frame])

zhanglei: [[0. 0.]
 [1. 1.]
 [2. 2.]]
[[ 0.99948355 -0.00212897 -0.03206386  0.36518713]
 [ 0.01229568 -0.89653382  0.44280461 -0.60687769]
 [-0.02968905 -0.44297017 -0.89604464  0.68282678]
 [ 0.          0.          0.          1.        ]]


In [ ]:
image = cv2.imread("/data/hdd1/storage/junpeng/ws_anygrasp/suctionnet-baseline/color2.png")
depth = np.load("/data/hdd1/storage/junpeng/ws_anygrasp/suctionnet-baseline/depth2.npy")
cloud = create_point_cloud_from_depth_image(depth,camera_info)

point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(cloud)
point_cloud.colors = o3d.utility.Vector3dVector(image.reshape(-1,3))
o3d.visualization.draw_geometries([base_frame, cam_frame, point_cloud])

# Visual Using plotly

In [ ]:
import plotly.graph_objects as go
import cv2
import numpy as np

# 模拟简单的图像和深度图数据
# 真实应用中，image和depth将是实际的图像和深度图
image = cv2.imread("/data/hdd1/storage/junpeng/ws_anygrasp/suctionnet-baseline/color2.png")
depth = np.load("/data/hdd1/storage/junpeng/ws_anygrasp/suctionnet-baseline/depth2.npy")
depth = depth/1000.

# 模拟的相机内参矩阵
intrinsics = np.array([601.1693115234375 , 0.,         637.8362426757812,
                    0. ,        600.8593139648438, 363.8018798828125,
                    0.,           0.,           1.        ]).reshape((3,3))
K = intrinsics

# 计算每个像素的XYZ坐标
height, width = depth.shape
X, Y = np.meshgrid(np.arange(width), np.arange(height))
Z = depth
X = (X - K[0, 2]) * Z / K[0, 0]
Y = (Y - K[1, 2]) * Z / K[1, 1]

# 将坐标和颜色值重塑为一维数组，以便于plotly处理
X = X.flatten()
Y = Y.flatten()
Z = Z.flatten()
colors = image.reshape(-1, 3) / 255.0  # 归一化颜色值

# 使用plotly创建三维散点图
kk = 3
fig = go.Figure(data=[go.Scatter3d(
    x=X[::kk], y=Y[::kk], z=Z[::kk],
    mode='markers',
    marker=dict(
        size=2,
        color=colors[::kk],                # 设置颜色
        opacity=0.8
    )
)])

# 设置图表布局
fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=0),
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

fig.show()

# TCP

In [ ]:
from scipy.spatial.transform import Rotation as R
import time

def z_axis_to_quaternion(z_axis):
    # 确保z_axis是单位向量
    z_axis = z_axis / np.linalg.norm(z_axis)
    
    # 选择一个默认上方向，计算x_axis
    y_axis_default = np.array([0, 1, 0])
    x_axis = np.cross(y_axis_default, z_axis)
    x_axis = x_axis / np.linalg.norm(x_axis)  # 归一化
    
    # 计算y_axis
    y_axis = np.cross(z_axis, x_axis)
    
    # 从轴构建旋转矩阵
    rotation_matrix = np.array([x_axis, y_axis, z_axis]).T
    
    # 将旋转矩阵转换为四元数
    quaternion = R.from_matrix(rotation_matrix).as_quat()  # [x, y, z, w]
    
    return quaternion